In [72]:
import numpy as np
import pandas as pd
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [93]:
df = pd.read_csv("version2.csv")

In [96]:
df

,Unnamed: 0,company,company_public_response,company_response,complaint_id,narratives,consumer_consent_provided,consumer_disputed,date_received,date_sent_to_company,...,state,sub_issue,sub_product,submitted_via,tags,timely,zip_code,Qtr,product_new,label
0,0,Experian Information Solutions Inc.,Company has responded to the consumer and the ...,Closed with explanation,2788944,There are many mistakes appear in my report wi...,Consent provided,NaN,2018-01-20,2018-01-20,...,MD,Account information incorrect,Credit reporting,Web,NaN,Yes,212XX,2018Q1,"Credit reporting, credit repair services, or o...","('Credit reporting, credit repair services, or..."
1,1,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",Company has responded to the consumer and the ...,Closed with explanation,2773990,I was pulling my credit to looking into buying...,Consent provided,NaN,2018-01-06,2018-01-06,...,FL,Information belongs to someone else,Credit reporting,Web,NaN,Yes,347XX,2018Q1,"Credit reporting, credit repair services, or o...","('Credit reporting, credit repair services, or..."
2,2,"Caliber Home Loans, Inc.",NaN,Closed with explanation,2737995,"Since XXXX XXXX, I was working with a loan con...",Consent provided,NaN,2017-11-26,2017-11-26,...,MD,NaN,Conventional home mortgage,Web,NaN,Yes,208XX,2017Q4,Mortgage,"('Mortgage', 'apply for mortgage')"
3,3,"Select Management Resources, LLC",Company believes it acted appropriately as aut...,Closed with non-monetary relief,2674806,On XX/XX/XXXX I went to NTL in Delaware for a ...,Consent provided,NaN,2017-09-15,2017-09-15,...,PA,NaN,Title loan,Web,NaN,Yes,190XX,2017Q3,Loan,"('loan', 'payment')"
4,4,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",Company has responded to the consumer and the ...,Closed with non-monetary relief,2683460,"Hello CFPB, I am sending this complaint to you...",Consent provided,NaN,2017-09-25,2017-09-25,...,MD,Their investigation did not fix an error on yo...,Credit reporting,Web,NaN,Yes,212XX,2017Q3,"Credit reporting, credit repair services, or o...","('Credit reporting, credit repair services, or..."
5,5,"EQUIFAX, INC.",NaN,Closed with explanation,2737599,I am a victim of identity theft and have 3 fra...,Consent provided,NaN,2017-11-26,2017-11-26,...,AZ,Information belongs to someone else,Credit reporting,Web,Servicemember,Yes,851XX,2017Q4,"Credit reporting, credit repair services, or o...","('Credit reporting, credit repair services, or..."
6,6,ALLY FINANCIAL INC.,Company has responded to the consumer and the ...,Closed with explanation,2767457,My husband and I requested a deferment through...,Consent provided,NaN,2017-12-29,2017-12-29,...,TX,Billing problem,Loan,Web,NaN,Yes,762XX,2017Q4,Loan,"('loan', 'Managing the loan or lease')"
7,7,JPMORGAN CHASE & CO.,NaN,Closed with explanation,2738007,"On XX/XX/XXXX, I opened a XXXX Checking accoun...",Consent provided,NaN,2017-11-26,2017-11-26,...,MO,Can't close your account,Checking account,Web,NaN,Yes,631XX,2017Q4,Checking or savings account,"('Checking or savings account', 'Closing an ac..."
8,8,Experian Information Solutions Inc.,Company has responded to the consumer and the ...,Closed with non-monetary relief,2719786,I filed a dispute with XXXX onXX/XX/XXXX confi...,Consent provided,NaN,2017-11-03,2017-11-03,...,GA,Account status incorrect,Credit reporting,Web,NaN,Yes,317XX,2017Q4,"Credit reporting, credit repair services, or o...","('Credit reporting, credit repair services, or..."
9,9,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",Company has responded to the consumer and the ...,Closed with explanation,2676373,This account XXXX XXXX is supposed to be close...,Consent provided,NaN,2017-09-17,2017-09-17,...,CA,Account status incorrect,Credit reporting,Web,Servicemember,Yes,922XX,2017Q3,"Credit reporting, credit repair services, or o...","('Credit reporting, credit repair services, or..."


In [98]:
z = {"Credit reporting, credit repair services, or other personal consumer reports":"Credit reporting",\
     "Money transfer, virtual currency, or money service":"Money service",\
     "Checking or savings account":"Checking or savings account",\
     "Loan":"Loan",\
     "Credit card or prepaid card":"Credit card or prepaid card",\
     "Debt collection":"Debt collection",\
     "Mortgage":"Mortgage"}
df['product_new'] = df.product_new.map(z)

In [99]:
text = df['narratives'].tolist()
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vect = TfidfVectorizer()
dtm = tfidf_vect.fit_transform(text)

In [100]:
dtm.shape

(87592, 53784)

In [101]:
prod_label = df['product_new'].tolist()
all_prod = list(set(prod_label))

In [102]:
len(prod_label)

87592

In [103]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import classification_report

X_train, X_test, y_train, y_test = train_test_split(dtm, prod_label,\
                                    test_size = 0.3, stratify = prod_label,\
                                                    random_state = 123)

In [104]:
from sklearn.model_selection import cross_validate
from sklearn import svm 

metrics = ['precision_macro','recall_macro','f1_macro']
clf = svm.LinearSVC().fit(X_train, y_train)
predicted = clf.predict(X_test)
precision, recall, fscore, support = precision_recall_fscore_support(y_test, predicted, labels = prod_label)
print(classification_report(y_test, predicted, target_names = all_prod))

                             precision    recall  f1-score   support

           Credit reporting       0.81      0.83      0.82      1582
Checking or savings account       0.80      0.80      0.80      2591
                       Loan       0.88      0.90      0.89     10680
Credit card or prepaid card       0.82      0.81      0.81      5404
            Debt collection       0.81      0.77      0.79      2618
              Money service       0.88      0.78      0.82       680
                   Mortgage       0.90      0.91      0.91      2723

                avg / total       0.85      0.85      0.85     26278



In [81]:
new_doc =[]
new_doc.append(str(input("input: ")))
X_new_tfidf = tfidf_vect.transform(list(new_doc))
#X_new_tfidf.shape
pred_new = clf.predict(X_new_tfidf)
pred_new

input: Received call from unknown debt collection agency on Wednesday XX/XX/XXXX at around XXXX XXXX saying they are attorneys and going to take me to court for wage garnishment for account 6 years old. Employee was rude and unprofessional. Employee used abusive and threatening language.


array(['Debt collection'], 
      dtype='<U76')

In [105]:
pred_new

array(['Debt collection'], 
      dtype='<U76')

#### Try for each unique product, analyze the issue

In [116]:
#For product "Credit reporting, credit repair services, or other personal consumer reports"
prod_1 = pred_new[0]
prod_1_label = df[df['product_new'] == "Loan"]['label'].tolist()
all_prod_1 = list(set(prod_1_label))

In [117]:
prod_1_text = df[df['product_new'] == "Loan"]['narratives'].tolist()
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vect_1 = TfidfVectorizer()
dtm_prod_1 = tfidf_vect_1.fit_transform(prod_1_text)

In [124]:
dtm_prod_1.shape
len(prod_1_label)

(8725, 20098)

8725

In [118]:
X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(dtm_prod_1, prod_1_label,\
                                    test_size = 0.3, stratify = prod_1_label,\
                                                    random_state = 123)

In [131]:
len(y_train_1)

6107

In [132]:
clf_1 = svm.LinearSVC().fit(X_train_1, y_train_1)
predicted_1 = clf_1.predict(X_test_1)
precision, recall, fscore, support = precision_recall_fscore_support(y_test_1, predicted_1, labels = prod_1_label)
print(classification_report(y_test_1, predicted_1, target_names = all_prod_1))

                                        precision    recall  f1-score   support

            ('loan', 'getting a loan')       0.58      0.49      0.53       265
('loan', 'Managing the loan or lease')       0.68      0.81      0.74       936
                    ('loan', 'others')       0.61      0.41      0.49       152
                    ('loan', 'report')       0.50      0.36      0.42       162
                   ('loan', 'payment')       0.47      0.10      0.17        69
           ('loan', 'enexpected fees')       0.58      0.61      0.59       870
                   ('loan', 'contact')       0.56      0.48      0.51       164

                           avg / total       0.61      0.62      0.61      2618



In [128]:
predicted_1.shape

(2618,)